In [1]:
# add signal information to existing Lima GMNS network created by DynusT_to_GMNS.R
import numpy as np
import pandas as pd
import os, shutil

In [23]:
# load network files that need to bie modified 
nodes = pd.read_csv('../../../examples/Lima/GMNS/node.csv')
links = pd.read_csv('../../../examples/Lima/GMNS/link.csv')
mvmts = pd.read_csv('../../../examples/Lima/GMNS/movement.csv')

In [3]:
# parse control.dat
f = open('../../../examples/Lima/source_network/control.dat', 'r')

In [4]:
# first line - number of timing plans
n_timePlans = int(f.readline())

In [5]:
timePlan_info = []
i = 0
while i < n_timePlans:
    timePlan_info.append(f.readline())
    i = i + 1
    
#TODO parse timePlan info
# Lima only has one, so handling manually for now

In [6]:
# node info
len(nodes)

2232

In [7]:
node_cols = {
    0: "node_id",
    1: "ctrl_type",
    2: "num_phases",
    3: "cycle_time"
}

phase_key = {
    1: "none",
    2: "yield",
    3: "4_stop",
    4: "signal",
    5: "signal",
    6: "2_stop"
}

In [8]:
rows = []
i = 0
while i < len(nodes):
    row = pd.Series(f.readline().split()).apply(int).rename(node_cols)
    row['ctrl_type'] = phase_key[row['ctrl_type']]
    rows.append(row)
    i = i + 1

ctrl_nodes = pd.DataFrame(rows)

In [9]:
sig_node_ct = len(ctrl_nodes[ctrl_nodes['ctrl_type'] == 'signal'])

i = 0
k = 0 
node_phase_lines = []
link_phase_lines = []
while i < sig_node_ct:
    # get first phase
    node_phase_line = f.readline()
    node_phase_lines.append(node_phase_line)
    # get num of phases assoc with that node
    node_id = int(node_phase_line.split()[0])
    node_phase_ct = int(ctrl_nodes[ctrl_nodes['node_id'] == node_id]['num_phases'])
    j = 0 
    inbound_link_ct = int(node_phase_line.split()[5])
    while j < inbound_link_ct:
        link_phase_line = f.readline()
        link_phase_lines.append(link_phase_line)
        j = j + 1
    k = k + 1
    if k >= node_phase_ct:
        i = i + 1
        k = 0

# TODO process into dfs

In [10]:
# skip next line "=======Two Way Stop Signs/Yield Signs Below ======="
f.readline()

'=======Two Way Stop Signs/Yield Signs Below =======\n'

In [11]:
# remainder is two-way stop/yield info
twoway_temp = f.readlines()

In [12]:
# EOF
f.close()

In [13]:
temp_df = pd.DataFrame([pd.Series(i.split()) for i in twoway_temp if twoway_temp.index(i) % 3 == 1]).astype('Int64')
temp_df

,0,1,2,3,4,5,6,7
0,100018,100019,100093,100019,100020,100019,<NA>,<NA>
1,100041,100040,100039,100040,100280,100040,<NA>,<NA>
2,100068,100069,100070,100069,100062,100069,100073,100069
3,100070,100071,100086,100071,100061,100071,100072,100071
4,100088,100086,100082,100086,100099,100086,100071,100086
...,...,...,...,...,...,...,...,...
173,104280,104286,104192,104286,104173,104286,103852,104286
174,104181,104322,104319,104322,104176,104322,104327,104322
175,104198,104327,104328,104327,104322,104327,104002,104327
176,104343,104339,103516,104339,104350,104339,104338,104339


In [14]:
links_twowaystop = pd.lreshape(temp_df, {'start_node': temp_df.columns[temp_df.columns % 2 == 0], 'end_node': temp_df.columns[temp_df.columns % 2 == 1]})
links_twowaystop

,start_node,end_node
0,100018,100019
1,100041,100040
2,100068,100069
3,100070,100071
4,100088,100086
...,...,...
675,103852,104286
676,104327,104322
677,104002,104327
678,104338,104339


In [163]:
## now modify the existing gmns tables

In [25]:
nodes['ctrl_type'] = nodes.merge(ctrl_nodes, on = 'node_id')['ctrl_type_y']
nodes

,node_id,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id
0,1,NaN,1523373.000,1003235.000,NaN,NaN,none,1,NaN
1,2,NaN,1523873.000,1003225.000,NaN,NaN,none,2,NaN
2,3,NaN,1524263.000,1003205.000,NaN,NaN,none,3,NaN
3,4,NaN,1524224.000,1002265.000,NaN,NaN,none,4,NaN
4,5,NaN,1523854.000,1002765.000,NaN,NaN,none,5,NaN
...,...,...,...,...,...,...,...,...,...
2227,104443,NaN,1531243.411,993559.625,NaN,NaN,none,62,NaN
2228,104444,NaN,1532299.387,993452.125,NaN,NaN,none,69,NaN
2229,104445,NaN,1507167.100,995978.919,NaN,NaN,signal,120,NaN
2230,104446,NaN,1507084.155,999960.306,NaN,NaN,none,192,NaN


In [26]:
# get control type for these -- two-way stop vs yield
# in Lima's case, it's all 2ways, no yields
links_twowaystop = links_twowaystop.merge(ctrl_nodes[['node_id', 'ctrl_type']], left_on='end_node', right_on='node_id')


In [28]:
# applying control types to movements
# generally, inherit ctrl_type of node
# except two-way stop/yield: use 'none' unless in links_twowaystop list

mvmts['ctrl_type'] = mvmts.merge(ctrl_nodes, on='node_id')['ctrl_type_y']
mvmts

,mvmt_id,node_id,name,ib_link_id,start_ib_lane,end_ib_lane,ob_link_id,start_ob_lane,end_ob_lane,type,penalty,capacity,ctrl_type
0,1,1,NaN,100002 1,1.0,1.0,1 100002,1.0,1.0,uturn,NaN,NaN,none
1,2,1,NaN,101990 1,1.0,1.0,1 101990,1.0,1.0,uturn,NaN,NaN,none
2,3,1,NaN,101997 1,1.0,1.0,1 101997,1.0,1.0,uturn,NaN,NaN,none
3,4,1,NaN,101998 1,1.0,1.0,1 101998,1.0,1.0,uturn,NaN,NaN,none
4,5,2,NaN,100004 2,1.0,1.0,2 100004,1.0,1.0,uturn,NaN,NaN,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12622,12623,104446,NaN,100739 104446,2.0,2.0,104446 192,1.0,1.0,right,NaN,NaN,none
12623,12624,104446,NaN,104445 104446,2.0,2.0,104446 118,1.0,1.0,right,NaN,NaN,none
12624,12625,104447,NaN,120 104447,1.0,1.0,104447 104445,1.0,1.0,thru,NaN,NaN,none
12625,12626,104447,NaN,104445 104447,1.0,1.0,104447 120,1.0,1.0,thru,NaN,NaN,none


In [29]:
links_twowaystop['link_id'] = links_twowaystop['start_node'].astype(str) + " " + links_twowaystop['end_node'].astype(str)
links_twowaystop

,start_node,end_node,node_id_x,ctrl_type_x,node_id_y,ctrl_type_y,link_id
0,100018,100019,100019,2_stop,100019,2_stop,100018 100019
1,100093,100019,100019,2_stop,100019,2_stop,100093 100019
2,100020,100019,100019,2_stop,100019,2_stop,100020 100019
3,100041,100040,100040,2_stop,100040,2_stop,100041 100040
4,100039,100040,100040,2_stop,100040,2_stop,100039 100040
...,...,...,...,...,...,...,...
675,104338,104339,104339,2_stop,104339,2_stop,104338 104339
676,103529,104402,104402,2_stop,104402,2_stop,103529 104402
677,104412,104402,104402,2_stop,104402,2_stop,104412 104402
678,103513,104402,104402,2_stop,104402,2_stop,103513 104402


In [57]:
# Lima dataset error: all links inbound to a two-way stop controlled intersection seem to be tagged as stop-controlled
# This is an empty df subset but normally would want to replace the ctrl_type of this list to be 'none'
mvmts[(~mvmts['ib_link_id'].isin(links_twowaystop['link_id'])) & (mvmts['node_id'].isin(ctrl_nodes[ctrl_nodes['ctrl_type'] == '2_stop']['node_id']))]

,mvmt_id,node_id,name,ib_link_id,start_ib_lane,end_ib_lane,ob_link_id,start_ob_lane,end_ob_lane,type,penalty,capacity,ctrl_type


In [70]:
# fix dtype of lanes
mvmts[[c for c in mvmts.columns if 'b_lane' in c]] = mvmts[[c for c in mvmts.columns if 'b_lane' in c]].astype('Int64')

In [60]:
# export current changes
nodes.to_csv('../../../examples/Lima/GMNS/node.csv', index=False)
links.to_csv('../../../examples/Lima/GMNS/link.csv', index=False)
mvmts.to_csv('../../../examples/Lima/GMNS/movement.csv', index=False)

In [81]:
ctrl_nodes[ctrl_nodes['ctrl_type'] == 'signal']

,node_id,ctrl_type,num_phases,cycle_time
449,100000,signal,2,188
450,100001,signal,2,188
452,100003,signal,3,226
455,100006,signal,2,188
459,100010,signal,2,188
...,...,...,...,...
2121,104313,signal,2,190
2125,104317,signal,2,190
2174,104373,signal,2,190
2196,104408,signal,2,190


In [89]:
## now create the new signal tables from node_phase_lines and link_phase_lines
# no coordination, one timing plan per controller simplifies things columns=["controller_id"]

signal_controllers = pd.DataFrame(ctrl_nodes[ctrl_nodes['ctrl_type'] == 'signal']['node_id']).rename(columns={"node_id": "controller_id"}).reset_index(drop=True)
signal_controllers

,controller_id
0,100000
1,100001
2,100003
3,100006
4,100010
...,...
160,104313
161,104317
162,104373
163,104408


In [75]:
# read as "timing plan 0 starts at 1740" (but what units??)
# not worrying about it for now because every controller in Lima data has one timing plan
timePlan_info

['    0.0 1740.0\n']

In [104]:
signal_timing_plans = pd.DataFrame(ctrl_nodes[ctrl_nodes['ctrl_type'] == 'signal'][['node_id', 'cycle_time']])
signal_timing_plans = signal_timing_plans.rename(columns = {'node_id': 'timing_plan_id', 'cycle_time': 'cycle_length'})
signal_timing_plans['controller_id'] = signal_timing_plans['timing_plan_id']
signal_timing_plans['cycle_length'] = signal_timing_plans['cycle_length']
signal_timing_plans['timeday_id'] = np.nan
signal_timing_plans['time_day'] = '11111111_0000_2359'

signal_timing_plans

,timing_plan_id,cycle_length,controller_id,timeday_id,time_day
449,100000,188,100000,NaN,11111111_0000_2359
450,100001,188,100001,NaN,11111111_0000_2359
452,100003,226,100003,NaN,11111111_0000_2359
455,100006,188,100006,NaN,11111111_0000_2359
459,100010,188,100010,NaN,11111111_0000_2359
...,...,...,...,...,...
2121,104313,190,104313,NaN,11111111_0000_2359
2125,104317,190,104317,NaN,11111111_0000_2359
2174,104373,190,104373,NaN,11111111_0000_2359
2196,104408,190,104408,NaN,11111111_0000_2359


In [105]:
signal_controllers.to_csv('../../../examples/Lima/GMNS/signal_controller.csv', index=False)
signal_timing_plans.to_csv('../../../examples/Lima/GMNS/signal_timing_plan.csv', index=False)